# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [ ]:
# import libraries
from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import udf, last, when, sum, mean, col, ceil, struct
import pyspark.sql.functions as F
from pyspark.sql.types import IntegerType, LongType, DoubleType
from pyspark.ml import Pipeline

from datetime import datetime

import numpy as np

import matplotlib.pyplot as plt
import re

In [ ]:
# create a Spark session
spark = SparkSession.builder \
        .master("local") \
        .appName("Sparkify") \
        .getOrCreate()
spark

# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

In [ ]:
# specify file path of the dataset
sparkify_data = "mini_sparkify_event_data.json"

In [ ]:
# read json file
df = spark.read.json(sparkify_data)
df.persist()

In [ ]:
# number of records in the dataset
size_orig = df.count()
print("The dataset has {} entries.".format(size_orig))

In [ ]:
# print the dataset schema
df.printSchema()

In [ ]:
# helper function for more convenience
def printCleandSize(size_clean, size_orig=size_orig):
    print("Dataset size after cleaning process: {} - removed {} entries".format(size_clean, size_orig - size_clean))

In [ ]:
# Drop NA-values in the user defined columns. 
'''
Values where NA-values must be prevented:
- userId -> identifying the user
- page -> which page was accessed: e.g. downgrade/upgrade/next song:
- registration: the date of the registration 
- ts -> play-time: needed for analyzing the user activity during the day 
- level -> free/paid: needed for analyzing the churn rate
- status -> HTTP status codes: check if user is unsatisfied with Sparkify's availability 
- method -> HTTP request type: check if user is unsatisfied with Sparkify's availability
- gender -> check if there is a different behaviour for different gender
- sessionId -> assures the session was valid

usefull but not necessary relevant:
- auth -> "Cancalled / Logged In"
- firstName -> first name of te user
- lastName -> last name of the user
- location -> user geographical location
- userAgent -> difference between Win/Mac users
- itemInSession

Depending in the page event NA-valus in the follwing columns are acceptable:
- song -> name of the played song
- artist -> name of the artist 
- length -> length of the song (do not remove NA's here or the column page only contains "next page")
'''
df_clean = df.dropna(how = "any", subset = ["userId", "sessionId", "method", "page", "ts", 
                                            "registration", "level", "userAgent", "method",
                                            "status"])
                           
size_clean = df_clean.count()
printCleandSize(size_clean)

In [ ]:
# cleaning the gender column - assuming the null values can be either null or a other gender
# later this groups will be converted into numerical values
df_clean = df_clean.fillna("null/other", subset=["gender"])
df_clean.groupby("gender").count().show()

In [ ]:
# check if after droping the NA-values there are still some invalid values
df_clean.select("userId").dropDuplicates().sort("userId").show()

In [ ]:
# filter userIds with an empty string
df_clean = df_clean.filter(df_clean["userId"] != "")
printCleandSize(df_clean.count())

In [ ]:
# user defined function to extract the play hour from the timestamp
get_hour = udf(lambda x: datetime.fromtimestamp(x / 1000.0).hour, IntegerType())
# create the a new column "hour" this holds the hour when a user interacted with the system
df_clean = df_clean.withColumn("hour", get_hour(df.ts))

In [ ]:
# side effect - the invalid usernames matched with the null/other values in the gender
df_clean.groupby("gender").count().show()

# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

### Preliminary Analysis
The following part covers a basic analysis of Sparkify's user data.

In [ ]:
# get the number of records
total_records = df_clean.count()
print("The dataset contains {} records.".format(total_records))

In [ ]:
# number of users
num_users = df_clean.select("userId").dropDuplicates().count()
print("The dataset contains {} users.".format(num_users))

#### User Groups (paid and unpaid memberships)
In this section the difference between paid and unpaid memberhips are shown. Since the users can change its membership status over time here all entries of the dataset are used.

In [ ]:
# get membership data
pd_user_level = df_clean.groupby("level").count().toPandas()
pd_user_level

In [ ]:
# plot memberhsip data
plt.figure();
plt.bar(pd_user_level.level.values, pd_user_level["count"], color=["lightgray", "darkgray"]);
plt.xlabel("Membership type");
plt.ylabel("Number of members");
plt.title("Sparkify's membership overview");

In [ ]:
# calculate the percentages
free_percentage = pd_user_level["count"][0] / total_records
paid_percentage = 1 - free_percentage
print("Membership overview:"+
      "\n free: {:.2f}% ({})\n paid: {:.2f}% ({})".format(free_percentage, pd_user_level["count"][0], 
                                                          paid_percentage, pd_user_level["count"][1]))

#### User groups: gender
Are Sparkify's user equally distributed between all genders:

In [ ]:
# get Sparkify users grouped by their gender
pd_gender = df_clean.groupby("gender").count().toPandas()
pd_gender.gender = pd_gender.gender.apply(lambda x: "female" if x == "F" else "male")

In [ ]:
plt.figure();
plt.bar(pd_gender["gender"], pd_gender["count"], color=["lightgray", "darkgray"]);
plt.ylabel("Number of users");
plt.xlabel("Gender");
plt.title("Spakify's users by gender");

In [ ]:
# percentage of female users
female_users = pd_gender["count"][0]/ total_records
female_users

In [ ]:
# percentage of male users
male_users = pd_gender["count"][1] / total_records
male_users

In [ ]:
# difference between female and male users
female_users - male_users

The amount of female users are a bit higher (+ 11%) than the amount of male users.

#### Playtime over the day 
Discover the playtime over the day in respect to different groups:
* all users
* gender
* payed / unpaid

In [ ]:
pd_playtime_all = df_clean.groupby("hour").count().sort("hour").toPandas()

In [ ]:
pd_playtime_paid = df_clean.where("level == 'paid'").groupby("hour").count().sort("hour").toPandas()

In [ ]:
pd_playtime_unpaid = df_clean.where("level != 'paid'").groupby("hour").count().sort("hour").toPandas()

In [ ]:
pd_playtime_men = df_clean.where("gender == 'M'").groupby("hour").count().sort("hour").toPandas()

In [ ]:
pd_playtime_female = df_clean.where("gender == 'F'").groupby("hour").count().sort("hour").toPandas()

In [ ]:
plt.figure(figsize=[20,10]);
plt.plot(pd_playtime_all["count"], label="all users");
plt.plot(pd_playtime_paid["count"], label="paid users");
plt.plot(pd_playtime_unpaid["count"], label="unpaid users");
plt.plot(pd_playtime_men["count"], label="usergroup: men");
plt.plot(pd_playtime_female["count"], label="female")
plt.xlabel("Hours during the day");
plt.ylabel("number of song plays per hour");
plt.title("Number of song plays during the day")
plt.legend();

#### Memberhship downgrades/upgrades
Exploring the membership up- and downgrades.

In [ ]:
# count page event for each entry in the log file
df_clean.groupby("page").count().show()

In [ ]:
# number of downgrades
num_downgrades = df_clean.where("page = 'Submit Downgrade'").count()
num_downgrades_unique =df_clean.where("page = 'Submit Downgrade'").dropDuplicates().count()
print("The total number of downgrades is {} from {} different users".format(num_downgrades,
                                                                            num_downgrades_unique))

In [ ]:
# number of upgrades
num_upgrades = df_clean.where("page = 'Submit Upgrade'").count()
num_upgrades_unique =df_clean.where("page = 'Submit Upgrade'").dropDuplicates().count()
print("The total number of upgrades is {} from {} different users".format(num_upgrades,
                                                                          num_upgrades_unique))

#### Location
This section covers a quick look at the location of Sparkify's users.

In [ ]:
# use only the last two characters from the location indicating the state
split_location = udf(lambda x: re.split(" ", x)[-1])

In [ ]:
pd_states = df_clean.dropDuplicates(["userId"]).withColumn("state", split_location(df_clean.location)).groupby("state").count().toPandas()

In [ ]:
plt.figure(figsize=[20,10]);
plt.bar(pd_states.state, pd_states["count"], color="green", alpha=0.8);
plt.xlabel("US state(s)");
plt.ylabel("Number of users");
plt.title("Sparkify's user loacation in the US");
plt.xticks(rotation=90);

In [ ]:
pd_states["count"].describe()

The users declared 58 different locations in the US - some location consits of several US states. Are are three areas with an higher user occurence.
* California on the west coast
* East coast area: New York, New Jersy and Pennsylvania
* South states with Texas and Florida

#### Artist and song
Most played artist and song:

In [ ]:
# most played artists
df_clean.where(col("artist").isNotNull()).groupby("artist").count().sort("count", ascending=False).show()

In [ ]:
# get the most played songs
df_clean.where(col("song").isNotNull()).groupby("song").count().sort("count", ascending=False).show()

In [ ]:
# get the artist of the most played song
df_clean.select(["artist", "song"]).filter(df_clean.song.like("You're The One")).dropDuplicates().show()

There are a lot of entries without an artist or song title, more precisely 50046 entries. These entries represent the page event except "Next song"

In [ ]:
# number of page events where the artist and song are null
num_page_event = total_records - df_clean.filter(df_clean.artist.isNotNull()).count()
num_page_event

In [ ]:
# number of page events where the artist is not null
num_artist_events = df.dropna(how = "any", subset = ["artist"]).groupby("page").count().head()[1]
num_artist_events

In [ ]:
# check if the artist and page events sum up to total record number -> true
num_artist_events + num_page_event == total_records

#### Technical Issuses
Take a look how often there are technical proplems on the Sparify platform - based on the http status codes

In [ ]:
pd_http_status = df_clean.groupby("status").count().toPandas()
pd_http_status

In [ ]:
plt.figure();
plt.bar(["307", "404", "200"] ,pd_http_status["count"], color=["lightyellow", "red", "lightgreen"]);

In [ ]:
print("Percentage of HTTP 404 Erros: {:.10f}".format(pd_http_status["count"][1]/ total_records))

Result: There is no need to care about technical issues. It seems that sparkify is a stable platform.

#### Registration time

In [ ]:
get_year = udf(lambda x: datetime.fromtimestamp(x / 1000.0).year, IntegerType())
# create the a new column "hour" this holds the hour when a user interacted with the system
df_clean.withColumn("year", get_year(df_clean.registration)).groupby("year").count().show()

In [ ]:
get_month = udf(lambda x: datetime.fromtimestamp(x / 1000).month, IntegerType())
get_day = udf(lambda x: datetime.fromtimestamp(x / 1000).day, IntegerType())

df_register = df_clean.withColumn("month", get_month(df_clean.registration))
df_register = df_register.withColumn("day", get_day(df_register.registration))
pd_reg_date = df_register.groupby(["month", "day"]).count().sort(["month", "day"]).toPandas()

In [ ]:
plt.figure();
plt.bar(pd_reg_date.month, pd_reg_date["count"]);
plt.xlabel("Month from January to December");
plt.ylabel("Number of registrations");
plt.title("Sparkify's user registration in 2018");

### Define Churn-Rate

In [ ]:
# user defined function for set a churn indicator
get_churn = udf(lambda x: 1 if x == "Cancellation Confirmation" else 0, IntegerType())

In [ ]:
# create new column churn
df_churn = df_clean.withColumn("churnEvent", get_churn(df_clean.page))

In [ ]:
# number of cancellations
num_cancellation = df_churn.where("churnEvent = 1").dropDuplicates(["userId"]).count()

In [ ]:
# calc churn rate
churn_rate = num_cancellation / num_users
print("The churn rate is at {:.3}%".format(churn_rate))

In [ ]:
# create window with userId - this is needed to extract the churn users
user_window = Window.partitionBy("userId")

In [ ]:
# create a new column churn, where a unique churn value is assigned to each user 
df_churn = df_churn.withColumn("churn", F.max("churnEvent").over(user_window))

In [ ]:
# get users grouped by churn
df_churn.dropDuplicates(["userid"]).groupby("churn").count().show()

#### Downgrade

In [ ]:
# user defined function for indicating a downgrade event
get_downgrade = udf(lambda x: 1 if x == "Downgrade" else 0, IntegerType())

In [ ]:
# create temporary downgradeEvent column
df_churn = df_churn.withColumn("downgradeEvent", get_downgrade(df_churn.page)) 

In [ ]:
# create new column downgrade 
df_churn = df_churn.withColumn("downgrade", F.max("downgradeEvent").over(user_window))

In [ ]:
# drop temporary columns
df_churn = df_churn.drop(*["downgradeEvent", "churnEvent"])

In [ ]:
# show how many users have downgraded their account from paid to unpaid
df_churn.dropDuplicates(["userId"]).groupby("downgrade").count().show()

### Data exploration

#### Churn in level (paid/unpaid)

In [ ]:
# show the account level (paid/unpaid) regarding the churn rate
df_churn.dropDuplicates(["userId"]).groupby(["churn", "level"]).count().sort("churn").show()

#### Churn in gender

In [ ]:
# show the gender of the users grouped by churn
df_churn.dropDuplicates(["userId"]).groupby(["churn", "gender"]).count().sort("churn").show()

#### Churn in user authentication 

In [ ]:
# check if the churn users were logged in 
df_churn.dropDuplicates(["userId"]).groupby(["churn", "auth"]).count().sort("churn").show()

#### Churn in location

In [ ]:
# get user location grouped by chrun
pd_states_churn = df_churn.withColumn("state", split_location(df_clean.location))\
                          .dropDuplicates(["userId"]).groupby(["churn", "state"])\
                          .count().sort("churn").toPandas()

In [ ]:
# unstack grouped variables for plotting the data
pd_states_plot = pd_states_churn.groupby(["state", "churn"]).sum().unstack().fillna(0)

In [ ]:
# create plot

# get required data
n_groups = pd_states_plot.shape[0]
user_state_churn = pd_states_plot["count"][1]
user_state = pd_states_plot["count"][0]
index_labels = pd_states_plot.index

# create plot
fig, ax = plt.subplots(figsize=[15,10])
index = np.arange(n_groups)
bar_width = 0.5
opacity = 0.8
plt.bar(index, user_state, bar_width, alpha=opacity, color="green", label="Non Churn Users")
plt.bar(index + bar_width, user_state_churn, bar_width, alpha=opacity, color="red", label="Churn Users")
plt.xlabel("US State(s)")
plt.ylabel("Number of users")
plt.title("Sparkify's user location")
plt.xticks(index + bar_width, index_labels, rotation=90)
plt.legend()
plt.tight_layout()
plt.show()

#### Time between registration and unsubscribe request

In [ ]:
# user defined function of calculating difference of two timestamps
udf_time = udf(lambda x: (x[1] - x[0]) if x[0] < x[1] else 0, IntegerType())

In [ ]:
# add new column "churntime" -> hours between registration and confirmed cancellation
df_churn_time = df_churn.where("page = 'Cancellation Confirmation'").select(["userId", "registration", "ts"])\
                        .withColumn("churntime", udf_time(struct('registration', 'ts')))\
                        .withColumn("churntime", F.abs(F.ceil(col("churntime")/1000/60/60))).drop("ts")

In [ ]:
# add the chruntime to matching users
df_churn = df_churn.join(df_churn_time, df_churn.userId == df_churn_time.userId, how='left')\
                   .select(df_churn["*"], df_churn_time["churntime"])

In [ ]:
# calculate descriptives
df_churn.dropDuplicates(["userID"]).select("churntime").describe().show()

In [ ]:
# fill NA chruntime values after looking at the desciptives
df_churn = df_churn.fillna(0, subset=["churntime"])

The average churn time is 307.17 hours, which equals about 12.8 days. The longest churn time is only 2 houers where in contrast the longest duration is about 23.45 days.  

#### Playtime

In [ ]:
# calculate playtime in minutes grouped by user churn
df_churn.where(df_churn.length.isNotNull()).groupby("length", "churn").count()\
.withColumn("time", (df_churn.length * col("count"))/60).groupby("churn").count().show()

In [ ]:
print("Total playtime for churn users: {:.2f} hours.".format(9195 / 60))
print("Total playtime for non-churn users: {:.2f} hours.".format(14336 / 60))

#### Page Roll Advert

In [ ]:
df_churn.where("page = 'Roll Advert'").groupby(["page", "churn"]).count().show()

#### Number of "Next Song" events

In [ ]:
df_churn.where("page = 'NextSong'").select(["page", "churn"]).groupby(["page", "churn"]).count().show()

#### Number of added Friends on Sparkify

In [ ]:
df_churn.where("page = 'Add Friend'").select(["userId", "page", "churn"]).groupby(["page", "churn"]).count().show()

#### Number of thumbs up / thumbs down

In [ ]:
df_churn.where("page = 'Thumbs Up' OR page = 'Thumbs Down'").groupby(["page", "churn"]).count().show()

#### Number of songs added to Playlist

In [ ]:
df_churn.where("page = 'Add to Playlist'").groupby(["page", "churn"]).count().show()

# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

#### Features
The following features are created in the following section:

Categorical features:
* Level (paid / unpaid membership)
* Gender
* Downgrade (user performed a downgrade)
* Location (state)
* Page (event)

Numerical features:
* Number of friends
* Number of Thumbs Up
* Number of Thumbs Down
* Number add to playlist
* Ratio like (Thumbs Up / Thumbs down)
* Number artists
* Number of songs per user
* Churntime (time from registation to cancelling event)

In [ ]:
from pyspark.sql.functions import avg, col, concat, count, desc, explode, lit, min, max, split, stddev, udf
from pyspark.ml.feature import CountVectorizer, IDF, Normalizer, PCA, RegexTokenizer, StandardScaler, StopWordsRemover, StringIndexer, VectorAssembler

#### Categorical Features

In [ ]:
def create_categorical_features(columns_list, df=df_churn):
    '''
    INPUT:
    columns_list - (list) column names of categorical features 
    df - (pyspark dataframe) dataframe
    
    OUTPUT:
    df - (pyspark dataframe) dataframe with categorical features
    
    Desciption:
    For each element in the columns list a categorical feature will
    be created using a StringIndexer.
    The original columns in columns_list will be replaced with the
    feature.
    '''
    for col in columns_list:
        indexer = StringIndexer(inputCol=col, outputCol=col+"Feat")
        df = indexer.fit(df).transform(df)
    return df.dropDuplicates(["userId"])

In [ ]:
# list of categorical features
category_feature_cols = ["gender", "level", "downgrade", "page"]
# create categorical features
df_features_cat = create_categorical_features(category_feature_cols).drop("churntime")

In [ ]:
df_features_cat.printSchema()

#### Numerical Features

In [ ]:
def get_features(df):
    '''
    INPUT: 
    df - (pyspark dataframe) cleaned dataframe
    
    OUTPUT:
    df_result - (pyspark dataframe) dataframe with numerical features
    
    Description:
    This function computes several numerical features:
    - friends: Number of friends per Sparkify user
    - dislike: Number of dislikes
    - like: Number of likes
    - like-ration: ratio between likes and dislikes
    - churn-time: time from registration to churn in days
    - avg-songs-in-session: average amount of songs a user played in a session
    - advert: number of advert
    '''
     
    df_friends = df.where("page = 'Add Friend'").groupby("userId").count().dropDuplicates(["userId"])\
                   .withColumn("numFriends", col("count")).drop("count")
        
    df_dislike = df.where("page = 'Thumbs Down'").groupby("userId").count().dropDuplicates(["userId"])\
                   .withColumn("numDislikes", col("count")).drop("count")

    df_like = df.where("page = 'Thumbs Up'").groupby("userId").count().dropDuplicates(["userId"])\
                .withColumn("numLikes", col("count")).drop("count")
    
    df_playlist = df.where("page = 'Add to Playlist'").groupby("userId").count().dropDuplicates(["userId"])\
                    .withColumn("playlistSize", col("count")).drop("count")
    
    df_likeRatio = df_dislike.join(df_like, on="userId", how="left")\
                             .withColumn("like-ratio", col("numLikes")/col("numDislikes"))
    
    df_churntime = df.select(["userId", "churntime"]).dropDuplicates(["userId"])
    
    df_time = df_churn.groupby("userId").agg(F.max("ts").alias("last_access"))
    ms_per_day = 86400000 # milliseconds per day
    df_time = df_time.join(df_churn, on="userId").withColumn("days", ((col("last_access")-col("registration"))/ms_per_day)\
                                                             .cast(IntegerType())).dropDuplicates(["userId"])\
                                                             .dropDuplicates(["userId"]).select("userId", "days")

    df_songs_in_session = df.groupby("userId", "ts").agg(max("itemInSession")).groupby("userId")\
                                                    .avg("max(itemInSession)")\
                                                    .withColumnRenamed("avg(max(itemInSession))", "avg_song_session")
    
    df_advert = df.where("page = 'Roll Advert'").groupby("userId").count().dropDuplicates(["userId"])\
                  .withColumn("numAdvert", col("count")).drop("count")
    
    df_result = df_friends.join(df_likeRatio, on="userId", how="inner")\
                          .join(df_playlist, on="userId", how="inner")\
                          .join(df_churntime, on="userId", how="inner")\
                          .join(df_time, on="userId", how="inner")\
                          .join(df_songs_in_session, on="userId", how="inner")\
                          .join(df_advert, on="userId", how="inner")
   
    
    return df_result

In [ ]:
# calculate numerical features
df_num_feat = get_features(df_churn)

In [ ]:
# a quick look into the numerical features
df_num_feat.show()

In [ ]:
# merge the numerical and categorical features
df_features = df_num_feat.join(df_features_cat, on="userId", how="inner")

In [ ]:
# print schema of merged dataframe
df_features.printSchema()

There are some useless columns in the dataframe.

In [ ]:
# drop all obsolete columns
df_features = df_features.drop("userId",
                               "artist",
                               "auth",
                               "firstName",
                               "gender",
                               "itemInSession",
                               "lastName",
                               "length",
                               "level",
                               "location",
                               "method", 
                               "page", 
                               "registration", 
                               "sessionId",
                               "song",
                               "status", 
                               "ts",
                               "userAgent",
                               "hour")

In [ ]:
# schema of cleaned dataframe
df_features.printSchema()

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

Classification problem
* Logistic Regression
* Decision Trees
* Gradient Boosting Trees
* SVM
* Naive Bayes

#### Create dataset for the Model
Convert the dataset ```df_features``` into a dataset which can directly be used in ML models using a Vector Assembler and Features Scaler (using StandardScaler).

In [ ]:
# list of numerical features
numerical_features = ["numFriends", "numDislikes", "numLikes", "like-ratio", "playlistSize", "churntime", "days", 
                      "avg_song_session", "numAdvert", "churn", "downgrade"]

# list of categorical features
categorical_features = ["genderFeat", "levelFeat", "downgradeFeat", "locationFeat", "pageFeat"]

In [ ]:
def create_process_pipeline(numerical_features = numerical_features, categorical_features = categorical_features):
    '''
    INPUT:
    numerical_features - (list) a list of strings naming the columns of numerical features
    categorical_features - (list) a list of strings naming the columns of categorical features
    
    OUTPUT:
    pipeline - (pyspark pipeline) a pipeline which includes all necessary processing steps, including a 
               VetorAssembler and a Standard Scaler
               
    Description:
    Build a pyspark pipeline for transforming the data in the desired layout.
    Important note: the categorical features were already converted using a String Indexer in a previous step.
    '''
    # numieric features: feature vectorizer and scaler 
    assembler_num = VectorAssembler(inputCols = numerical_features, 
                                    outputCol = "numerical_features")
    
    feature_scaler_num = StandardScaler(withMean = True, withStd = True, 
                                   inputCol = "numerical_features", 
                                   outputCol = "scaled_features")

    # categorical features: 
    assembler_cat = VectorAssembler(inputCols = categorical_features+["scaled_features"], 
                                    outputCol = "features")
    
    # create pipeline
    pipeline = Pipeline(stages = [assembler_num, feature_scaler_num, assembler_cat])
    
    return pipeline

In [ ]:
def preprocess_data(df_features, numerical_features = numerical_features, categorical_features = categorical_features):
    '''
    INPUT:
    df - (pyspark dataframe) a dataframe which includes all features
    numerical_features - (list) a list of strings naming the columns of numerical features
    categorical_features - (list) a list of strings naming the columns of categorical features
    
    OUTPUT:
    df - (pyspark dataframe) which includes all columns from the input dataset, but extendet 
         with the features in an additional representation (defined with pipeline)
    '''
    # create the pipeline
    pipeline = create_process_pipeline(numerical_features, categorical_features)

    # fit and transform the data
    df = pipeline.fit(df_features).transform(df_features)
    
    # 
    df = df.withColumnRenamed("churn", "label")
    
    return df

In [ ]:
# preprocess the data 
df_model = preprocess_data(df_features, numerical_features, categorical_features)

#### Model creation

In [ ]:
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, GBTClassifier, LinearSVC
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator


In [ ]:
def create_and_test_model(classifier, test, train):

    if classifier == "LogisticRegression":
        ml = LogisticRegression()
    elif classifier == "RandomForestClassifier":
        ml = RandomForestClassifier()
    elif classifier == "GBTClassifier":
        ml = GBTClassifier()
    elif classifier == "LinearSVC":
        ml = LinearSVC()
    else:
        return "Error: Invalid model."
    
    # fit (train) model
    model = ml.fit(train)

    # make prediction for unknown test data
    predictions = model.transform(test)
    
    # Select (prediction, true label) and compute test error
    # based on pyspark documentation: https://spark.apache.org/docs/latest/ml-classification-regression.html
    # https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.evaluation.MulticlassClassificationEvaluator.html
    evaluator = MulticlassClassificationEvaluator(metricName = "f1",
                                                  labelCol = "label",
                                                  predictionCol = "prediction")
    
    f1_score = evaluator.evaluate(predictions)
    print("Classifier: {} - F1-Score: {:.2f}".format(classifier, f1_score))
    
    auc_evaluator = BinaryClassificationEvaluator()
    metric_value = auc_evaluator.evaluate(predictions, {auc_evaluator.metricName: "areaUnderROC"})
    print("The areaUnderROC on the test set is {:.2%}".format(metric_value))
    
    return f1_score
    

In [ ]:
# split the model data into train and test-set
train, test = df_model.randomSplit([0.8, 0.2], seed=42)

In [ ]:
def find_best_model(train, test, classifier=["LogisticRegression", "RandomForestClassifier", "GBTClassifier", "LinearSVC"]):
    
    for clf in classifier:
        create_and_test_model(clf, test, train)
        

In [ ]:
find_best_model(train, test, ["RandomForestClassifier", "GBTClassifier", "LinearSVC"])

# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.